In [24]:
!pip install openai PyPDF2 nbformat nbconvert
!pip install nbconvert --upgrade
!pip uninstall crewai crewai_tools -y
!pip install crewai crewai_tools
!pip install groq

Found existing installation: crewai 0.41.1
Uninstalling crewai-0.41.1:
  Successfully uninstalled crewai-0.41.1
Found existing installation: crewai-tools 0.4.26
Uninstalling crewai-tools-0.4.26:
  Successfully uninstalled crewai-tools-0.4.26
  Using cached crewai-0.41.1-py3-none-any.whl.metadata (13 kB)
  Using cached crewai_tools-0.4.26-py3-none-any.whl.metadata (4.6 kB)
Using cached crewai-0.41.1-py3-none-any.whl (91 kB)
Using cached crewai_tools-0.4.26-py3-none-any.whl (84 kB)


In [17]:
!pip install openai PyPDF2 nbformat nbconvert crewai python-dotenv

import os
from dotenv import load_dotenv
import openai
import PyPDF2
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool, BaseTool

# Load environment variables from .env file
load_dotenv()

# Environment variables for OpenAI API Key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

# Define tools as classes inheriting from BaseTool with proper type annotations and docstrings
@tool
class OpenAIApiTool(BaseTool):
    """Tool to query the OpenAI API."""
    name: str = "openai_api_tool"
    description: str = "Tool to query the OpenAI API."

    def __call__(self, prompt: str) -> str:
        response = openai.Completion.create(
            engine="davinci",
            prompt=prompt,
            max_tokens=100
        )
        return response.choices[0].text.strip()

@tool
class FileReaderTool(BaseTool):
    """Tool to read and extract text from PDF files."""
    name: str = "file_reader_tool"
    description: str = "Tool to read and extract text from PDF files."

    def __call__(self, file_path: str) -> str:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfFileReader(file)
            content = ""
            for page_num in range(reader.numPages):
                page = reader.getPage(page_num)
                content += page.extract_text()
        return content

@tool
class FileRunnerTool(BaseTool):
    """Tool to run Jupyter notebooks and return the output."""
    name: str = "file_runner_tool"
    description: str = "Tool to run Jupyter notebooks and return the output."

    def __call__(self, notebook_path: str) -> dict:
        with open(notebook_path) as f:
            nb = nbformat.read(f, as_version=4)
        
        ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
        ep.preprocess(nb, {'metadata': {'path': './'}})
        
        return nb

# Instantiate tools
openai_api_tool = OpenAIApiTool()
file_reader_tool = FileReaderTool()
file_runner_tool = FileRunnerTool()

# Define agents
data_analyst = Agent(
    role='Data Analyst',
    goal='Read and summarize PDF files.',
    backstory='You have expertise in extracting meaningful insights from large documents.',
    tools=[openai_api_tool, file_reader_tool]
)

financial_analyst = Agent(
    role='Financial Analyst',
    goal='Run financial forecasting using Jupyter notebooks.',
    backstory='You excel at using Python notebooks to derive financial insights.',
    tools=[openai_api_tool, file_runner_tool]
)

# Define tasks
read_and_summarize_pdf_task = Task(
    description='Read the "merged_Quant.pdf" file and summarize its content.',
    expected_output='A summarized report highlighting the key points from the PDF.',
    tools=[openai_api_tool, file_reader_tool],
    agent=data_analyst
)

run_financial_forecasting_task = Task(
    description='Run the "SPY-Financial Forecasting OHLC.ipynb" Jupyter notebook and generate a report.',
    expected_output='A detailed report generated from the Jupyter notebook.',
    tools=[openai_api_tool, file_runner_tool],
    agent=financial_analyst
)

# Create crew
crew = Crew(
    agents=[data_analyst, financial_analyst],
    tasks=[read_and_summarize_pdf_task, run_financial_forecasting_task],
    process=Process.sequential
)

# Kickoff the crew
result = crew.kickoff(inputs={})
print(result)


TypeError: 'Tool' object is not callable

In [27]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from groq import Groq
import yaml

# Load environment variables from .env file
load_dotenv()

# Reading API keys from environment variables
# serper_api_key = os.getenv("SERPER_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

# Initialize Groq client
groq_client = Groq(api_key=groq_api_key)

# Paths to YAML files
agents_yaml_path = './agents.yaml'
tasks_yaml_path = './tasks.yaml'

# Define a function to read YAML files
def read_yaml(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

# Load agents and tasks from YAML files
agents_data = read_yaml(agents_yaml_path)
tasks_data = read_yaml(tasks_yaml_path)

# Multi-Model Configuration
def create_agent(data, model="groq"):
    return Agent(
        role=data['role'],
        goal=data['goal'],
        backstory=data['backstory'],
        verbose=True,
        memory=True,
        model=model,
        client=groq_client if model == "groq" else None
    )

# Create agents and assign models
agents = [
    create_agent(agent, model="openai-gpt-4") if key in ["portfolio_manager", "quant_finance_teacher"]
    else create_agent(agent, model="groq")
    for key, agent in agents_data.items()
]

# Create tasks
tasks = [Task(**task) for task in tasks_data.values()]

# Form the crew
crew = Crew(
    agents=agents,
    tasks=tasks,
    process=Process.sequential
)

# Kickoff the crew
result = crew.kickoff()
print(result)


ValidationError: 1 validation error for Task
agent
  Input should be a valid dictionary or instance of Agent [type=model_type, input_value='data_analyst', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/model_type

In [23]:
!dir

 Volume in drive C is Acer
 Volume Serial Number is 8A0E-B286

 Directory of c:\Users\Administrator\Desktop\DataSciencePortfolio\QuantitativeFinance\CrewAI

08/01/2024  12:25 PM    <DIR>          .
08/01/2024  11:38 AM    <DIR>          ..
08/01/2024  12:26 PM               268 .env
08/01/2024  12:23 PM             2,157 agents.yaml
08/01/2024  12:06 PM           162,724 crew.ipynb
08/01/2024  12:25 PM             1,649 tasks.yaml
               4 File(s)        166,798 bytes
               2 Dir(s)  379,461,672,960 bytes free
